In [1]:
import pandas as pd

df = pd.read_json("hf://datasets/allenai/real-toxicity-prompts/prompts.jsonl", lines=True)

input_dicts = df["prompt"].values.flatten().tolist()

inputs = [d["text"] for d in input_dicts]

In [ ]:
inputs

In [3]:
# from src.perspective.backend import PerspectiveBackend
# from src.perspective.utils import api_key_from_env, api_key_from_file

# api_key = api_key_from_file("PERSPECTIVE_API_KEY.txt")
# backend = PerspectiveBackend(api_key=api_key, rate_limit=1)

In [4]:
from src.detoxify.backend import DetoxifyBackend

backend = DetoxifyBackend(model_type="original", amp=False)

In [5]:
from src.rating import *

rating_runner = RatingRunner(backend)
results = rating_runner.rate_stram(inputs[-200:], batch_size=100)

In [ ]:
results_list = list(results)

In [ ]:
toxicity_scores = [r.scores["toxicity"] for r in results_list]

# Plot the distribution of toxicity scores
import matplotlib.pyplot as plt

plt.hist(toxicity_scores, bins=20)

In [ ]:
# Print examples of toxicity > 0.5 and < 0.5
toxic_examples = [r for r in results_list if r.scores["toxicity"] > 0.5]
non_toxic_examples = [r for r in results_list if r.scores["toxicity"] <= 0.5]

print("Toxic examples:")
for r in toxic_examples[:5]:
    print(r.text)

print("Non-toxic examples:")
for r in non_toxic_examples[:5]:
    print(r.text)

In [2]:
from src.hf_auto_model.backend import AutoModelBackend
from huggingface_hub import login
from src.perspective.utils import api_key_from_file

hf_key = api_key_from_file("HF_KEY.txt")
login(token=hf_key)

backend = AutoModelBackend()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from src.generation import GenerationRunner

generation_runner = GenerationRunner(backend)
generated_texts = generation_runner.generate_stream(inputs[:10])

In [ ]:
generated_list = list(generated_texts)

# Print the generated texts as well as the input prompts
for generated_text in generated_list:
    if not generated_text.error:
        print("Input prompt:")
        print(generated_text.prompt)
        print("Generated text:")
        print(generated_text.outputs)
        print()
    else:
        print("Error:", generated_text.error)

In [5]:
generated_text = generation_runner.generate_single("This is a test prompt.")

print(generated_text.outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[' Please do not submit it as a real answer.\n\n## Step 1: Understand the task\nThe task is to provide a solution to a problem that does not exist yet.\n\n## Step 2: Determine the problem\nSince the problem does not exist']
